In [1]:
import os
from getpass import getpass
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install cohere langchain openai tiktoken -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.4/794.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver do

In [3]:
!wget https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py

--2023-12-28 17:06:01--  https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10203 (10.0K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   9.96K  --.-KB/s    in 0s      

2023-12-28 17:06:01 (44.0 MB/s) - ‘utils.py’ saved [10203/10203]



In [4]:
# Если используете ключ из курса, запустите эту ячейку
from utils import ChatOpenAI

#course_api_key= "Введите ваш OpenAI API ключ"
course_api_key = getpass(prompt='Введите API ключ')

# инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)

Введите API ключ··········


In [5]:
import pandas as pd

df = pd.read_csv('https://stepik.org/media/attachments/lesson/1110806/100_crypto_messages.csv')

In [6]:
df.sample(5)

,text_id,text
6,39,Привествую! Могу помочь тебе обучиться зарабат...
80,49,"Биpжeвoй apбитpaж, ищy пoмoщникoв, вceмy наyчy..."
1,179,Что вы думаете о концепции децентрализованных ...
96,200,Какие криптовалюты вы видите в будущем как осн...
26,295,"FUD (Fear, Uncertainty, Doubt) - это негативны..."


In [17]:
from langchain import PromptTemplate, FewShotPromptTemplate
from langchain.prompts import ChatPromptTemplate
examples = [
    {
        "text_example": "Привествую! Могу помочь тебе обучиться зарабатывать. Могу показать тему которая принесет 100-200$ каждый день. Провожу набор, всего 6 бойцов! Научим абсолютно БЕСПЛАТНО.От тебя только желание и телефон с меня обучение и доведение до результата. Напиши по контактам, что бы приобрести больше",
        "answer": "1"
    }, {
        "text_example": "Что вы думаете о концепции децентрализованных бирж (DEX)? Какие преимущества они предоставляют по сравнению с традиционными биржами?",
        "answer": "0"
    }
]
example_template = """text: {text_example}
{answer}
"""

# создаём промпт из шаблона выше
example_prompt = PromptTemplate(
    input_variables=["text_example", "answer"],
    template=example_template)

prefix = """Представь что ты участник чата на тематику криптовалюты.
В чате присутствуют как полезные советы и информация про криптовалюту, так и спам в виде сомнительных предложений заработка не связанных с криптовалютой
или просто нерелевантные сообщения. Ты хочешь определить какие сообщения являются реальными и полезными, а какие бесполезный мусор. Из следующего текста определи:
Является ли сообщение спамом?
Ответь 1 если является и 0 если нет.

Вот несколько примеров:
"""
suffix = """
text: {text}

Answer:
"""
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["text"],
    example_separator="\n\n"
)

In [9]:
print(few_shot_prompt_template)

input_variables=['text'] examples=[{'text_example': 'Привествую! Могу помочь тебе обучиться зарабатывать. Могу показать тему которая принесет 100-200$ каждый день. Провожу набор, всего 6 бойцов! Научим абсолютно БЕСПЛАТНО.От тебя только желание и телефон с меня обучение и доведение до результата. Напиши по контактам, что бы приобрести больше', 'answer': '1'}, {'text_example': 'Что вы думаете о концепции децентрализованных бирж (DEX)? Какие преимущества они предоставляют по сравнению с традиционными биржами?', 'answer': '0'}] example_prompt=PromptTemplate(input_variables=['answer', 'text_example'], template='text: {text_example}\nAI: {answer}\n') suffix='\ntext: {text}\n\nAnswer: \n' prefix='Представь что ты участник чата на тематику криптовалюты.\nВ чате присутствуют как полезные советы и информация про криптовалюту, так и спам в виде сомнительных предложений заработка не связанных с криптовалютой\nили просто нерелевантные сообщения. Ты хочешь определить какие сообщения являются реальн

In [11]:
df.head()

,text_id,text,is_spam
0,143,Удаленная работа. От нас: Зп от 2500$,1
1,179,Что вы думаете о концепции децентрализованных ...,1
2,147,Удалённый заработок. Время 1-2часа в день. Мож...,1
3,157,Меня всегда интересовал вопрос безопасности хр...,1
4,168,"Я слышал, что некоторые банки начали экспериме...",1


In [18]:
answers = []
for index, row in df.iterrows():
  print('Index', index, end=' ')
  text = row["text"]
  ans = llm.predict(few_shot_prompt_template.format(text=text))
  print(ans)
  answers.append(int(ans))


Index 0 1
Index 1 0
Index 2 1
Index 3 0
Index 4 0
Index 5 0
Index 6 1
Index 7 0
Index 8 0
Index 9 0
Index 10 0
Index 11 1
Index 12 0
Index 13 0
Index 14 1
Index 15 1
Index 16 0
Index 17 0
Index 18 1
Index 19 0
Index 20 0
Index 21 1
Index 22 0
Index 23 0
Index 24 0
Index 25 0
Index 26 0
Index 27 1
Index 28 1
Index 29 0
Index 30 1
Index 31 0
Index 32 1
Index 33 1
Index 34 1
Index 35 1
Index 36 0
Index 37 0
Index 38 1
Index 39 1
Index 40 0
Index 41 1
Index 42 1
Index 43 0
Index 44 1
Index 45 1
Index 46 1
Index 47 0
Index 48 1
Index 49 0
Index 50 0
Index 51 1
Index 52 0
Index 53 1
Index 54 1
Index 55 0
Index 56 1
Index 57 1
Index 58 1
Index 59 0
Index 60 1
Index 61 1
Index 62 0
Index 63 0
Index 64 1
Index 65 1
Index 66 1
Index 67 1
Index 68 0
Index 69 1
Index 70 1
Index 71 1
Index 72 1
Index 73 0
Index 74 0
Index 75 1
Index 76 0
Index 77 1
Index 78 0
Index 79 0
Index 80 1
Index 81 0
Index 82 0
Index 83 0
Index 84 1
Index 85 0
Index 86 0
Index 87 0
Index 88 1
Index 89 0
Index 90 0
Index 91 

In [19]:
df['is_spam'] = answers

In [20]:
df.head(10)

,text_id,text,is_spam
0,143,Удаленная работа. От нас: Зп от 2500$,1
1,179,Что вы думаете о концепции децентрализованных ...,0
2,147,Удалённый заработок. Время 1-2часа в день. Мож...,1
3,157,Меня всегда интересовал вопрос безопасности хр...,0
4,168,"Я слышал, что некоторые банки начали экспериме...",0
5,276,Майнинг Bitcoin потребляет много электроэнерги...,0
6,39,Привествую! Могу помочь тебе обучиться зарабат...,1
7,275,Концепция криптовалюты была предложена в 1998 ...,0
8,267,Какие аспекты экологии майнинга криптовалюты с...,0
9,207,Учите анализ рынка. Знание графиков и индикато...,0


In [22]:
df.to_csv("submission_100_rows", index=False, columns=['text_id', 'is_spam'])